<a href="https://colab.research.google.com/github/Y4S1-Deep-Learning/Apple-Leaves-Desease-Detection---Supervised-Learning/blob/kaushalya/Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Mask RCNN**

In [ ]:
!git clone https://github.com/akTwelve/Mask_RCNN.git

In [ ]:
import os
import sys
os.chdir('Mask_RCNN/samples')

In [ ]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

**Install libraries**

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from Mask_RCNN.mrcnn.utils import Dataset
from matplotlib import pyplot
from Mask_RCNN.mrcnn.visualize import display_instances
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.config import Config
from Mask_RCNN.mrcnn import utils

**Load training dataset**

In [ ]:
#images_path = "dataset/images.zip"
#annotation_path = "dataset/annots.zip"

#!unzip(os.path.join("content", images_path), "/content/dataset/train/images")
!unzip "/content/dataset/images.zip" -d "/content/dataset/train/"
!unzip "/content/dataset/annots.zip" -d "/content/dataset/train/"

In [ ]:
class AppleLeafDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define classes
        self.add_class("dataset", 1, "Black Rot")
        self.add_class("dataset", 2, "Apple Scab")
        self.add_class("dataset", 3, "Cedar Apple")

        # define data locations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'


		# find all images
        for filename in listdir(images_dir):
            print(filename)
			# extract image id
            image_id = filename[:-4]
			#print('IMAGE ID: ',image_id)

			# skip all images after 115 if we are building the train set
            if is_train and int(image_id) >= 250:
                continue
			# skip all images before 115 if we are building the test/val set
            if not is_train and int(image_id) < 250:
                continue
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2,3])


	# extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
		# load and parse the file
        tree = ElementTree.parse(filename)
		# get the root of the document
        root = tree.getroot()
		# extract each bounding box
        boxes = list()
        for box in root.findall('.//object'):
            name = box.find('name').text   #Add label name to the box list
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
		# extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

	# load the masks for an image
    def load_mask(self, image_id):
		# get details of image
        info = self.image_info[image_id]
		# define box file location
        path = info['annotation']
        #return info, path


		# load XML
        boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]


            # box[4] will have the name of the class
            if (box[4] == 'Black Rot'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('Black Rot'))
            elif(box[4] == 'Apple Scab'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('Apple Scab'))
            elif(box[4] == 'Cedar Apple'):
                masks[row_s:row_e, col_s:col_e, i] = 3
                class_ids.append(self.class_names.index('Cedar Apple'))

        return masks, asarray(class_ids, dtype='int32')


	# load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
dataset_dir = "/content/dataset/train"
# train set
train_set = AppleLeafDataset()
train_set.load_dataset(dataset_dir, is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test/val set
test_set = AppleLeafDataset()
test_set.load_dataset(dataset_dir, is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

import random
num=random.randint(0, len(train_set.image_ids))
# define image id
image_id = num
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

**Training**

In [ ]:
# define a configuration for the model
class AppleLeavesConfig(Config):
	# define the name of the configuration
	NAME = "object"

	NUM_CLASSES = 1 + 3
	# number of training steps per epoch
	STEPS_PER_EPOCH = 10

# prepare config
config = AppleLeavesConfig()
config.display()

In [ ]:
# Directory to save logs and trained model
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
import keras
print(keras.__version__)

In [ ]:
!pip install --upgrade keras    #upgrade keras

In [ ]:
from Mask_RCNN.mrcnn.model import MaskRCNN

from keras import models
model = models.Sequential()

In [ ]:
# define the model
model = MaskRCNN(mode='training', model_dir="logs", config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights("/content/mask_rcnn_coco.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])


# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=20, layers='heads')

In [ ]:
from matplotlib.patches import Rectangle